In [ ]:
import tensorflow as tf ### models
import numpy as np
import pandas as pd ### reading and procesing data
import seaborn as sns ### visualizacion
from tensorflow import keras
import matplotlib.pyplot as plt
import pydot as pydot
import tensorflow_datasets as tfds
from plot_model import plot_model
from tensorflow.keras.layers import Normalization, Dense, InputLayer, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization 
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam

In [ ]:
"""
#DEFINIMOS LA TAREA:

En este caso crearemos un modelo para clasificar si una celula posee malaria o no. Para ello utilizaremos una red neuronal convolucional.
La malaria es una enfermedad causada por un parásito Plasmodium, el cual es trasmitido por la picadura de un mosquito infectado. 
Sólo el género anófeles del mosquito transmite la malaria. Los síntomas de esta enfermedad pueden incluir fiebre, vómito y/o dolor de cabeza.
Las muestras de sangre son examinadas con un microscopio para diagnosticar la malaria, en donde el parásito es detectado dentro de los glóbulos rojos.
Ante esto estamos frente a una clasificacion binaria ya que nuestro paciente solo podra tener la enfermedad o no tenerla

En las muestras de las celulas nos daremos cuenta si estas estan infectadas por el color. 
Utilizamos una imagen, de una determinada cantidad de pixeles. Estaremos haciendo una clasificacion de imagenes.
"""

In [ ]:
"""
DATA PREPARATION:

El conjunto de datos de Malaria contiene un total de 27.558 imágenes de células con instancias iguales de células parasitadas y no infectadas 
de las imágenes de diapositivas de frotis de sangre delgada de células segmentadas.
La altura de pixeles varia un poco, por lo que no tienen todas las imagenes la misma cantidad de pixeles.
"""

dataset, dataset_info = tfds.load('malaria', with_info = True, as_supervised=True, shuffle_files=True, split=['train'])

In [ ]:
def split(dataset, TRAIN_RATIO, VAL_RATIO):
    DATA_SIZE = len(dataset)
    #TRAIN SET
    train_dataset = dataset.take(int(TRAIN_RATIO*DATA_SIZE))

    #VAL SET
    val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATA_SIZE))
    val_dataset = dataset.take(int(TRAIN_RATIO*DATA_SIZE))

    #TEST SET
    test_dataset = val_test_dataset.skip(int(TRAIN_RATIO*DATA_SIZE))

    return train_dataset, val_dataset, test_dataset

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset, val_dataset, test_dataset = split(dataset[0], TRAIN_RATIO, VAL_RATIO)




In [ ]:
"""VISUALIZACION"""
for i, (image,label) in enumerate(train_dataset.take(16)):
    ax = plt.subplot(4,4,i+1)
    plt.imshow(image)
    plt.title(dataset_info.features['label'].int2str(label))
    plt.axis('off')

In [ ]:
IM_SIZE = 224
def resize_rescale(image, label):
    return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

In [ ]:
"""
DATA PROCESSING

Vamos a procesar el tamaño de nuestras imagenes para luego normalizarlo
En nuestro caso normalizaremos a 224x224 pero en algunos casos se puede estandarizar el tamaño de los pixeles.
"""

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)


In [ ]:
BATCH_SIZE = 32

train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(1)
#No nos hace falta hacerlo para el test set ya que no necesitamos mezclarlo ni premezclarlo, pero si para el train y el validation set.
#Esto se hace para que tengan las mismas dimensiones.

In [ ]:
"""
Para este ejercicio utilizaremos redes neuronales convulcionales o CNN. Las CNN se utilizan para el procesar imagenes.
"""

In [ ]:
lenet_model = keras.Sequential([
    InputLayer(shape = (IM_SIZE, IM_SIZE, 3)),

    Conv2D(filters = 6, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu'),
    BatchNormalization(), #Normalizamos estandarizando -> (X - Media) \ desviacion estandar
    MaxPool2D(pool_size=2, strides=2),

    Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2), #Max pool usa el valor maximo del pool en cuestion

    Flatten(),

    Dense(100, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'relu'),
    BatchNormalization(),
    Dense(1, activation = 'sigmoid'), #Solo tenemos un output ya que puede estar o no infectada

])

lenet_model.summary()

In [ ]:
#Compilamos nuestro modelo
lenet_model.compile(
    optimizer= Adam(learning_rate=0.01),
    loss = BinaryCrossentropy(),
    metrics = 'accuracy' #Nos mostrara la precision del modelo
    )

In [ ]:
#Entrenamos nuestro modelo
history = lenet_model.fit(train_dataset, validation_data=val_dataset, epochs=100, verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train_loss', 'val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train_accuracy', 'val_accuracy'])
plt.show()

In [ ]:
lenet_model.evaluate(test_dataset)

In [ ]:
def parasite_or_not(x):
    if (x<0.5):
        return str('P')
    else:
        return str('U')

In [ ]:
for i, (image, label) in enumerate(test_dataset.take(9)):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(image[0])
    plt.title(str(parasite_or_not(label.numpy()[0]) + ":" + str(parasite_or_not(lenet_model.predict(image)[0][0]))))

    plt.axis('off')